In [ ]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import scipy
from PIL import Image
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops
from cnn_utils import *
# from test_utils import summary, comparator

%matplotlib inline
np.random.seed(1)

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_happy_dataset()

# normalization
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [ ]:
# Example 

index = 124
plt.imshow(X_train_orig[index])
plt.show()

# Sequential API

In [ ]:
"""
the forward propagation for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> FLATTEN -> DENSE
"""

def happyModel():

    model = tf.keras.Sequential([
            
            tfl.ZeroPadding2D(padding=(3, 3), input_shape=(64, 64, 3)),
    
            tfl.Conv2D(32, (7, 7), strides=(1,1)),
    
            tfl.BatchNormalization(axis=3),
        
            tfl.ReLU(),
        
            tfl.MaxPool2D(),
        
            tfl.Flatten(),
        
            tfl.Dense(1, activation='sigmoid')
        ])
    
    return model

In [ ]:
happy_model = happyModel()

happy_model.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

In [ ]:
happy_model.summary()

In [ ]:
# train the model

happy_model.fit(X_train, Y_train, epochs=10, batch_size=16)

In [ ]:
# evaluate the model

happy_model.evaluate(X_test, Y_test)

## Functional API

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_signs_dataset()

In [ ]:
# Example 
index = 9
plt.imshow(X_train_orig[index])
print ("y = " + str(np.squeeze(Y_train_orig[:, index])))

In [ ]:
X_train = X_train_orig/255.
X_test = X_test_orig/255.
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [ ]:
"""
    the forward propagation for the model:
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> DENSE
"""

def convolutional_model(input_shape):

    input_img = tf.keras.Input(shape=input_shape)
    
    Z1 = tfl.Conv2D(8, (4,4), padding='same')(input_img)
    A1 = tfl.ReLU()(Z1)
    P1 = tfl.MaxPool2D(pool_size=(8,8), strides=8, padding='same')(A1)
    
    Z2 = tfl.Conv2D(16, (2,2), padding='same')(P1)
    A2 = tfl.ReLU()(Z2)
    P2 = tfl.MaxPool2D(pool_size=(4,4), strides=4, padding='same')(A2)
    
    F = tfl.Flatten()(P2)
    
    outputs = tfl.Dense(6, activation="softmax")(F)
    
    
    model = tf.keras.Model(inputs=input_img, outputs=outputs)
    
    
    return model

In [ ]:
conv_model = convolutional_model((64, 64, 3))

conv_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
conv_model.summary()

In [ ]:
# train the model

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(64)
history = conv_model.fit(train_dataset, epochs=100, validation_data=test_dataset)

In [ ]:
history.history

In [ ]:
# visualisation of history

df_loss_acc = pd.DataFrame(history.history)
df_loss= df_loss_acc[['loss','val_loss']]
df_loss.rename(columns={'loss':'train','val_loss':'validation'},inplace=True)
df_acc= df_loss_acc[['accuracy','val_accuracy']]
df_acc.rename(columns={'accuracy':'train','val_accuracy':'validation'},inplace=True)
df_loss.plot(title='Model loss',figsize=(12,8)).set(xlabel='Epoch',ylabel='Loss')
df_acc.plot(title='Model Accuracy',figsize=(12,8)).set(xlabel='Epoch',ylabel='Accuracy')